In [1]:
import os
import csv
import json
import pandas as pd
from collections import Counter, defaultdict

It's doing better than before on shapes.. and it can now build diamonds sometimes.. When looking at the visualizer, I saw it messing up square as rectangle quite a bit. Also, for diamonds the size was always 3 for the examples I saw. Can you check with the visualizer that when we detect a square it's actually a square and for diamonds it's making of varied sizes? Also, we need the orientation breakdown.

In [45]:
home=%pwd
# csv_path = home + '/llama_synth_level_one_output.csv'
csv_path = home + '/llama_aug_synth_level_one_output_for_analysis.csv'

In [4]:
# with open(csv_path, newline='') as f:
#     reader = csv.reader(f)
#     data = list(reader)

In [3]:
df = pd.read_csv(csv_path)

In [4]:
df.shape

(1218, 7)

In [46]:
df.head(10)

,Shape,Color,Shape_corr,Size_corr,Loc_corr,Orient_corr,Full_struct_corr
0,cube,1,1,1,1.0,NaN,1
1,tower,1,1,1,1.0,NaN,1
2,tower,1,1,1,1.0,NaN,1
3,tower,1,1,1,1.0,NaN,1
4,tower,1,1,1,1.0,NaN,1
5,tower,1,1,1,1.0,NaN,1
6,tower,1,1,1,1.0,NaN,1
7,tower,1,1,1,1.0,NaN,1
8,diamond,1,1,0,NaN,NaN,0
9,diamond,1,0,0,NaN,NaN,0


In [6]:
Counter(df['Shape_corr'].tolist())

Counter({1: 994, 0: 224})

In [7]:
Counter(df['Size_corr'].tolist())

Counter({1: 881, 0: 337})

In [8]:
Counter(df['Color'].tolist())

Counter({1: 1209, 0: 9})

In [9]:
Counter(df['Full_struct_corr'].tolist())

Counter({1: 631, 0: 587})

In [ ]:
#Counter(df['Loc_corr'].tolist())

In [10]:
#which samples didn't get color right? Only when the moves were botched
df.loc[df['Color'] == 0]

,Shape,Color,Shape_corr,Size_corr,Loc_corr,Orient_corr,Full_struct_corr
670,rectangle_botched,0,0,0,0.0,0.0,0
720,rectangle_botched,0,0,0,0.0,0.0,0
767,rectangle_botched,0,0,0,0.0,0.0,0
768,rectangle_botched,0,0,0,0.0,0.0,0
862,rectangle_botched,0,0,0,0.0,0.0,0
863,rectangle_botched,0,0,0,0.0,0.0,0
910,rectangle_botched,0,0,0,0.0,0.0,0
1052,rectangle_botched,0,0,0,0.0,0.0,0
1147,rectangle_botched,0,0,0,0.0,0.0,0


In [11]:
#what the breakdown of correct and incorrect shapes?
#Correct:
Counter(df.loc[df['Shape_corr'] == 1, 'Shape'].tolist())

Counter({'tower': 504,
         'row': 168,
         'diagonal': 113,
         'rectangle': 83,
         'square': 78,
         'diamond': 25,
         'cube': 23})

In [14]:
#Incorrect:
Counter(df.loc[df['Shape_corr'].isin([0,1]), 'Shape'].tolist())

Counter({'tower': 504,
         'diagonal': 168,
         'row': 168,
         'square': 144,
         'diamond': 108,
         'rectangle': 93,
         'cube': 24,
         'rectangle_botched': 9})

In [34]:
#what the breakdown of correct and incorrect sizes?
#Correct:
Counter(df.loc[df['Size_corr'] == 0, 'Shape'].tolist())

Counter({'diamond': 107,
         'diagonal': 79,
         'rectangle': 70,
         'square': 66,
         'rectangle_botched': 9,
         'cube': 6})

In [16]:
#what the breakdown of correct and incorrect locations?
#Correct:
Counter(df.loc[df['Loc_corr'] == 1, 'Shape'].tolist())

Counter({'tower': 212,
         'square': 62,
         'row': 43,
         'rectangle': 18,
         'cube': 12,
         'diagonal': 3})

In [13]:
Counter(df[(df['Size_corr'] == 1)&(df['Shape_corr'] == 1)]['Shape'].tolist())

Counter({'tower': 504,
         'row': 168,
         'diagonal': 89,
         'square': 78,
         'rectangle': 23,
         'cube': 18,
         'diamond': 1})

In [30]:
Counter(df[(df['Loc_corr'].isin([1,0]))&(df['Shape_corr'] == 1)]['Shape'].tolist())

Counter({'tower': 378,
         'row': 126,
         'diagonal': 80,
         'rectangle': 60,
         'square': 53,
         'cube': 18})

In [31]:
Counter(df[(df['Loc_corr'] == 1)&(df['Shape_corr'] == 1)]['Shape'].tolist())

Counter({'tower': 250,
         'row': 78,
         'rectangle': 40,
         'square': 33,
         'diagonal': 25,
         'cube': 12})

In [34]:
Counter(df[(df['Orient_corr'].isin([1,0]))&(df['Shape_corr'] == 1)]['Shape'].tolist())

Counter({'square': 60, 'rectangle': 59, 'diamond': 13})

In [36]:
Counter(df[(df['Orient_corr'].isin([1]))&(df['Shape_corr'] == 1)]['Shape'].tolist())

Counter({'square': 60, 'rectangle': 59, 'diamond': 13})

In [39]:
#get orientation types
indices = df.index[(df['Orient_corr'].isin([1]))&(df['Shape_corr'] == 1)].tolist()

In [48]:
json_path = home + '/llama_aug_synth_function_output.json'

with open(json_path, 'r') as j:
    jfile = json.load(j)
    samples = jfile

In [14]:
orient_dict = defaultdict(list)

for sample in samples:
    shape = sample['shape'] 
    if sample['index'] in indices:
        orient_dict[shape].append(sample['orient'])
        

In [16]:
Counter(orient_dict['rectangle'])

Counter({'horizontal': 17, 'vertical': 14})

In [17]:
Counter(orient_dict['square'])

Counter({'horizontal': 50, 'vertical': 11})

#### for each incorrect square, get the instruction size, then check if square

In [20]:
import functions as fn

In [63]:
rec_squares = [['index', 'inst_size', 'rec_size']]

In [50]:
#get squares that are wrong because they are rectangles -- maybe try "is rectangle" in these cases
indices = df.index[df['Shape'].isin(['square'])& df['Shape_corr'].isin([0])].tolist()

In [51]:
len(indices)

66

In [52]:
indices[:3]

[14, 15, 40]

In [17]:
# json_path = home + '/llama_synth_function_output.json'

# with open(json_path, 'r') as j:
#     jfile = json.load(j)
#     samples = jfile

In [64]:
for ind in indices:
    shape = samples[ind]['shape']
    inst = samples[ind]['size']
    seq = samples[ind]['net_seq']
    rec_size = None
    is_rectu = fn.is_rectangle_unfilled(seq)
    is_rect = fn.is_rectangle(seq)
    if is_rect:
        rec_size = is_rect[1]
    elif is_rectu:
        rect_size = is_rectu[1]
    rec_squares.append([ind, inst, rec_size])



In [65]:
rec_squares[:3]

[['index', 'inst_size', 'rec_size'],
 [14, [4, 4], (5, 4)],
 [15, [5, 5], (6, 5)]]

In [68]:
import csv
square_path = home + '/rec_squares.csv'

In [69]:
with open(square_path, 'w', newline='') as cfile:
    writer = csv.writer(cfile)
    writer.writerows(rec_squares)